In [ ]:
import pandas as pd
import numpy as np
from matplotlib import cm
from scipy.interpolate import griddata
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('logs/analyseParameters.csv')
df.head()

In [ ]:
# Setting up the plot style
sns.set(style="whitegrid")

# Creating subplots for each feature against `acc_sound`
fig, axs = plt.subplots(1, 3, figsize=(18, 6))  # 3 subplots in a row for `seq_len`, `gen_len`, `mem_len`

# Plotting `acc_sound` changes with `seq_len` grouped by `temp`
sns.lineplot(data=df, x='seq_len', y='acc_sound', hue='temp', marker='o', ax=axs[0], errorbar=None)
axs[0].set_title("acc_sound vs seq_len by temp")
axs[0].set_xlabel("seq_len")
axs[0].set_ylabel("acc_sound")

# Plotting `acc_sound` changes with `gen_len` grouped by `temp`
sns.lineplot(data=df, x='gen_len', y='acc_sound', hue='temp', marker='o', ax=axs[1], errorbar=None)
axs[1].set_title("acc_sound vs gen_len by temp")
axs[1].set_xlabel("gen_len")
axs[1].set_ylabel("acc_sound")

# Plotting `acc_sound` changes with `mem_len` grouped by `temp`
sns.lineplot(data=df, x='mem_len', y='acc_sound', hue='temp', marker='o', ax=axs[2], errorbar=None)
axs[2].set_title("acc_sound vs mem_len by temp")
axs[2].set_xlabel("mem_len")
axs[2].set_ylabel("acc_sound")

# Adjusting layout for clarity
plt.tight_layout()
plt.legend(title="temp", loc='upper right', bbox_to_anchor=(1.2, 1))
plt.show()

In [ ]:
# Setting up the plot style
sns.set(style="whitegrid")

# Creating subplots for each feature against `acc_sound`
fig, axs = plt.subplots(1, 3, figsize=(18, 6))  # 3 subplots in a row for `seq_len`, `gen_len`, `mem_len`

# Plotting `acc_sound` changes with `seq_len` grouped by `temp`
sns.lineplot(data=df, x='seq_len', y='acc_delta', hue='temp', marker='o', ax=axs[0], errorbar=None)
axs[0].set_title("acc_delta vs seq_len by temp")
axs[0].set_xlabel("seq_len")
axs[0].set_ylabel("acc_sound")

# Plotting `acc_sound` changes with `gen_len` grouped by `temp`
sns.lineplot(data=df, x='gen_len', y='acc_delta', hue='temp', marker='o', ax=axs[1], errorbar=None)
axs[1].set_title("acc_delta vs gen_len by temp")
axs[1].set_xlabel("gen_len")
axs[1].set_ylabel("acc_sound")

# Plotting `acc_sound` changes with `mem_len` grouped by `temp`
sns.lineplot(data=df, x='mem_len', y='acc_delta', hue='temp', marker='o', ax=axs[2], errorbar=None)
axs[2].set_title("acc_delta vs mem_len by temp")
axs[2].set_xlabel("mem_len")
axs[2].set_ylabel("acc_sound")

# Adjusting layout for clarity
plt.tight_layout()
plt.legend(title="temp", loc='upper right', bbox_to_anchor=(1.2, 1))
plt.show()


In [ ]:

# Step 1: Group by 'gen_len' and 'mem_len' and calculate the mean of 'acc_sound' for unique pairs
df_grouped = df.groupby(['gen_len', 'mem_len'], as_index=False)['acc_sound'].mean()

# Step 2: Define a high-resolution grid for smoother interpolation
gen_len_vals = np.linspace(df_grouped['gen_len'].min(), df_grouped['gen_len'].max(), 100)
mem_len_vals = np.linspace(df_grouped['mem_len'].min(), df_grouped['mem_len'].max(), 100)
X, Y = np.meshgrid(gen_len_vals, mem_len_vals)

# Step 3: Interpolate acc_sound values onto the high-resolution grid
Z = griddata(
    (df_grouped['gen_len'], df_grouped['mem_len']),
    df_grouped['acc_sound'],
    (X, Y),
    method='cubic'
)

# Step 4: Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plotting the surface with smoother color transitions
surf = ax.plot_surface(X, Y, Z, cmap=cm.viridis, edgecolor='none', rstride=1, cstride=1, antialiased=True)

# Adding color bar for reference
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Setting labels
ax.set_xlabel("gen_len")
ax.set_ylabel("mem_len")
ax.set_zlabel("acc_sound")
ax.set_title("acc_sound vs gen_len and mem_len")

# Show plot
plt.show()


In [ ]:
# Step 1: Group by 'gen_len' and 'mem_len' and calculate the mean of 'acc_sound' for unique pairs
df_grouped = df.groupby(['gen_len', 'mem_len'], as_index=False)['acc_delta'].mean()

# Step 2: Define a high-resolution grid for smoother interpolation
gen_len_vals = np.linspace(df_grouped['gen_len'].min(), df_grouped['gen_len'].max(), 100)
mem_len_vals = np.linspace(df_grouped['mem_len'].min(), df_grouped['mem_len'].max(), 100)
X, Y = np.meshgrid(gen_len_vals, mem_len_vals)

# Step 3: Interpolate acc_sound values onto the high-resolution grid
Z = griddata(
    (df_grouped['gen_len'], df_grouped['mem_len']),
    df_grouped['acc_delta'],
    (X, Y),
    method='cubic'
)

# Step 4: Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plotting the surface with smoother color transitions
surf = ax.plot_surface(X, Y, Z, cmap=cm.viridis, edgecolor='none', rstride=1, cstride=1, antialiased=True)

# Adding color bar for reference
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Setting labels
ax.set_xlabel("gen_len")
ax.set_ylabel("mem_len")
ax.set_zlabel("acc_delta")
ax.set_title("acc_delta vs gen_len and mem_len")

# Show plot
plt.show()

In [ ]:
# Create a parallel coordinates plot
fig = px.parallel_coordinates(df, dimensions=['seq_len', 'gen_len', 'mem_len', 'acc_sound'],
                              color="acc_sound", color_continuous_scale=px.colors.sequential.Viridis)
fig.show()


In [ ]:
# Create a parallel coordinates plot
fig = px.parallel_coordinates(df, dimensions=['seq_len', 'gen_len', 'mem_len', 'acc_delta'],
                              color="acc_delta", color_continuous_scale=px.colors.sequential.Viridis)
fig.show()


In [ ]:
# Create 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
x=df['seq_len'],
y=df['gen_len'],
z=df['mem_len'],
mode='markers',
marker=dict(
    size=5,
    color=df['acc_sound'],  # Color by `acc_sound`
    colorscale='Viridis',
    colorbar=dict(title='acc_sound')
)
)])
fig.update_layout(scene=dict(
xaxis_title='seq_len',
yaxis_title='gen_len',
zaxis_title='mem_len'),
title='3D Scatter Plot of acc_sound')
fig.show()


In [ ]:
# Create 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=df['seq_len'],
    y=df['gen_len'],
    z=df['mem_len'],
    mode='markers',
    marker=dict(
        size=5,
        color=df['acc_delta'],  # Color by `acc_sound`
        colorscale='Viridis',
        colorbar=dict(title='acc_delta')
    )
)])
fig.update_layout(scene=dict(
    xaxis_title='seq_len',
    yaxis_title='gen_len',
    zaxis_title='mem_len'),
    title='3D Scatter Plot of acc_delta')
fig.show()

In [ ]:
# Set up figure for subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Heatmap for seq_len vs. gen_len, aggregated over mem_len
pivot_seq_gen = df.groupby(['seq_len', 'gen_len'])['acc_sound'].mean().unstack()
sns.heatmap(pivot_seq_gen, ax=axes[0], cmap="viridis")
axes[0].set_title('acc_sound by seq_len and gen_len')
axes[0].set_xlabel('gen_len')
axes[0].set_ylabel('seq_len')
axes[0].invert_yaxis()  # Start y-axis from lowest to highest

# Heatmap for seq_len vs. mem_len, aggregated over gen_len
pivot_seq_mem = df.groupby(['seq_len', 'mem_len'])['acc_sound'].mean().unstack()
sns.heatmap(pivot_seq_mem, ax=axes[1], cmap="viridis")
axes[1].set_title('acc_sound by seq_len and mem_len')
axes[1].set_xlabel('mem_len')
axes[1].set_ylabel('seq_len')
axes[1].invert_yaxis()  # Start y-axis from lowest to highest

# Heatmap for gen_len vs. mem_len, aggregated over seq_len
pivot_gen_mem = df.groupby(['gen_len', 'mem_len'])['acc_sound'].mean().unstack()
sns.heatmap(pivot_gen_mem, ax=axes[2], cmap="viridis")
axes[2].set_title('acc_sound by gen_len and mem_len')
axes[2].set_xlabel('mem_len')
axes[2].set_ylabel('gen_len')
axes[2].invert_yaxis()  # Start y-axis from lowest to highest

plt.tight_layout()
plt.show()



In [ ]:
# Set up figure for subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Heatmap for seq_len vs. gen_len, aggregated over mem_len
pivot_seq_gen = df.groupby(['seq_len', 'gen_len'])['acc_delta'].mean().unstack()
sns.heatmap(pivot_seq_gen, ax=axes[0], cmap="viridis")
axes[0].set_title('acc_delta by seq_len and gen_len')
axes[0].set_xlabel('gen_len')
axes[0].set_ylabel('seq_len')
axes[0].invert_yaxis()  # Start y-axis from lowest to highest

# Heatmap for seq_len vs. mem_len, aggregated over gen_len
pivot_seq_mem = df.groupby(['seq_len', 'mem_len'])['acc_delta'].mean().unstack()
sns.heatmap(pivot_seq_mem, ax=axes[1], cmap="viridis")
axes[1].set_title('acc_delta by seq_len and mem_len')
axes[1].set_xlabel('mem_len')
axes[1].set_ylabel('seq_len')
axes[1].invert_yaxis()  # Start y-axis from lowest to highest

# Heatmap for gen_len vs. mem_len, aggregated over seq_len
pivot_gen_mem = df.groupby(['gen_len', 'mem_len'])['acc_delta'].mean().unstack()
sns.heatmap(pivot_gen_mem, ax=axes[2], cmap="viridis")
axes[2].set_title('acc_delta by gen_len and mem_len')
axes[2].set_xlabel('mem_len')
axes[2].set_ylabel('gen_len')
axes[2].invert_yaxis()  # Start y-axis from lowest to highest

plt.tight_layout()
plt.show()
